# Chapter 16 – Natural Language Processing with RNNs and Attention

This notebook contains all the sample code and solutions to the exercises in chapter 16.

## Setup
This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

*Warning*: the latest TensorFlow versions are based on Keras 3. For previous chapters, it wasn't too hard to update the code to support Keras 3, but unfortunately it's much harder for this chapter: for example, stateful RNNs work very differently, ragged tensors are no longer supported, TensorFlow Hub models are no longer supported, and more. So for this chapter I've had to revert to Keras 2. To do that, I set the TF_USE_LEGACY_KERAS environment variable to "1" and import the tf_keras package. This ensures that tf.keras points to tf_keras, which is Keras 2.*.

In [2]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" 

# import tf_keras

In [3]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

As we did in earlier chapters, let's define the default font sizes to make the figures prettier:

In [4]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

And let's create the ```images/nlp``` folder (if it doesn't already exist), and define the ```save_fig()``` function which is used through this notebook to save the figures in high-res for the book:

In [5]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "nlp"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

This chapter can be very slow without a GPU, so let's make sure there's one, or else issue a warning:

In [6]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

# Generating Shakespearean Text Using a Character RNN
## Creating the Training Dataset
Let's download the Shakespeare data from Andrej Karpathy's char-rnn project

In [7]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [8]:
# extra code – shows a short text sample
print(shakespeare_text[:80])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [9]:
# extra code – shows all 39 distinct characters (after converting to lower case)
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [10]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [11]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [12]:
n_tokens

39

In [13]:
dataset_size

1115394

In [14]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [15]:
# extra code – a simple example using to_dataset()
# There's just one sample in this dataset: the input represents "to b" and the
# output represents "o be"
list(to_dataset(text_vec_layer(["To be"])[0], length=4))

[(<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 4,  5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 5,  2, 23,  3]], dtype=int64)>)]

In [16]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

## Building and Training the Char-RNN Model
Warning: the following code may one or two hours to run, depending on your GPU. Without a GPU, it may take over 24 hours. If you don't want to wait, just skip the next two code cells and run the code below to download a pretrained model.

Note: the GRU class will only use cuDNN acceleration (assuming you have a GPU) when using the default values for the following arguments: activation, recurrent_activation, recurrent_dropout, unroll, use_bias and reset_after.

In [17]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [18]:
# tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.Dense(n_tokens, activation="softmax")
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
#               metrics=["accuracy"])
# model_ckpt = tf.keras.callbacks.ModelCheckpoint(
#     "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
# history = model.fit(train_set, validation_data=valid_set, epochs=10,
#                     callbacks=[model_ckpt])

In [19]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

If you don't want to wait for training to complete, I've pretrained a model for you. The following code will download it. Uncomment the last line if you want to use it instead of the model trained above.

In [20]:
# extra code – downloads a pretrained model
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz", url, extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakespeare_model = tf.keras.models.load_model(model_path)

In [21]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 3s 3s/step


'e'

## Generating Fake Shakespearean Text

In [22]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]], dtype=int64)>

In [23]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [24]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [25]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

In [26]:
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 25ms/step
To be or not to be the duke
as it is a proper strange death,
and the


In [27]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 21ms/step
To be or not to begg, i
point and through you are to-morrow?

romeo:


In [28]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 23ms/step
To be or not to bepevicm-vilv!?$mz?gmjz :3?ljb'va;!td&
i.ur3l'-j!3em


## Stateful RNN

In [29]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [30]:
# extra code – simple example using to_dataset_for_stateful_rnn()
list(to_dataset_for_stateful_rnn(tf.range(10), 3))

[(<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[0, 1, 2]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]])>),
 (<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[3, 4, 5]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[4, 5, 6]])>),
 (<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[6, 7, 8]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[7, 8, 9]])>)]

If you'd like to have more than one window per batch, you can use the ```to_batched_dataset_for_stateful_rnn()``` function instead of ```to_dataset_for_stateful_rnn()```:

In [31]:
# extra code – shows one way to prepare a batched dataset for a stateful RNN

import numpy as np

def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]])>)]

In [32]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])


In [33]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [34]:
# extra code – use a different directory to save the checkpoints
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_stateful_shakespeare_model",
    monitor="val_accuracy",
    save_best_only=True)

**Warning**: the following cell will take a while to run (possibly an hour if you are not using a GPU).

In [35]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
   9996/Unknown - 64s 6ms/step - loss: 1.8697 - accuracy: 0.4492

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 67s 7ms/step - loss: 1.8695 - accuracy: 0.4492 - val_loss: 1.6977 - val_accuracy: 0.4920
Epoch 2/10
9991/9999 [============================>.] - ETA: 0s - loss: 1.5677 - accuracy: 0.5273

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


...

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 67s 7ms/step - loss: 1.3906 - accuracy: 0.5725 - val_loss: 1.5422 - val_accuracy: 0.5387
Epoch 9/10
9999/9999 [==============================] - 65s 6ms/step - loss: 1.3839 - accuracy: 0.5738 - val_loss: 1.5417 - val_accuracy: 0.5364
Epoch 10/10
9997/9999 [============================>.] - ETA: 0s - loss: 1.3792 - accuracy: 0.5750

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 71s 7ms/step - loss: 1.3792 - accuracy: 0.5750 - val_loss: 1.5369 - val_accuracy: 0.5410


**Extra Material: converting the stateful RNN to a stateless RNN and using it**

To use the model with different batch sizes, we need to create a stateless copy:

In [36]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

To set the weights, we first need to build the model (so the weights get created):

In [37]:
stateless_model.build(tf.TensorShape([None, None]))

In [38]:
stateless_model.set_weights(model.get_weights())

In [39]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

In [40]:
tf.random.set_seed(42)

print(extend_text("to be or not to be", temperature=0.01))

1/1 [==============================] - 0s 21ms/step
to be or not to be
a wife is a strange her hand and her hands,
and i


## Sentiment Analysis

In [41]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [42]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


In [43]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

**Warning**: the following cell will take a few minutes to run and the model will probably not learn anything because we didn't mask the padding tokens (that's the point of the next section).

In [44]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 21s 28ms/step - loss: 0.6935 - accuracy: 0.4981 - val_loss: 0.6931 - val_accuracy: 0.5016
Epoch 2/2
704/704 [==============================] - 19s 27ms/step - loss: 0.6928 - accuracy: 0.5055 - val_loss: 0.6942 - val_accuracy: 0.5004


## Masking
**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [45]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 22s 29ms/step - loss: 0.6289 - accuracy: 0.6395 - val_loss: 0.6051 - val_accuracy: 0.7028
Epoch 2/5
704/704 [==============================] - 20s 28ms/step - loss: 0.4288 - accuracy: 0.8064 - val_loss: 0.3727 - val_accuracy: 0.8360
Epoch 3/5
704/704 [==============================] - 20s 28ms/step - loss: 0.3351 - accuracy: 0.8561 - val_loss: 0.4518 - val_accuracy: 0.8024
Epoch 4/5
704/704 [==============================] - 20s 28ms/step - loss: 0.3030 - accuracy: 0.8741 - val_loss: 0.3097 - val_accuracy: 0.8684
Epoch 5/5
704/704 [==============================] - 20s 28ms/step - loss: 0.2788 - accuracy: 0.8846 - val_loss: 0.3233 - val_accuracy: 0.8576


Or using manual masking:

In [46]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

Warning: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [47]:
# extra code – compiles and trains the model, as usual
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 23s 30ms/step - loss: 0.5395 - accuracy: 0.7136 - val_loss: 1.2668 - val_accuracy: 0.6256
Epoch 2/5
704/704 [==============================] - 20s 28ms/step - loss: 0.3805 - accuracy: 0.8356 - val_loss: 0.3277 - val_accuracy: 0.8640
Epoch 3/5
704/704 [==============================] - 20s 28ms/step - loss: 0.3025 - accuracy: 0.8746 - val_loss: 0.3158 - val_accuracy: 0.8632
Epoch 4/5
704/704 [==============================] - 20s 29ms/step - loss: 0.2800 - accuracy: 0.8872 - val_loss: 0.3053 - val_accuracy: 0.8688
Epoch 5/5
704/704 [==============================] - 21s 29ms/step - loss: 0.2621 - accuracy: 0.8924 - val_loss: 0.3120 - val_accuracy: 0.8668


**Extra material: using ragged tensors**

In [48]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

In [49]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [50]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5


c:\Users\schre\anaconda3\envs\tf_dev\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


704/704 [==============================] - 23s 31ms/step - loss: 0.6034 - accuracy: 0.6708 - val_loss: 0.5328 - val_accuracy: 0.7440
Epoch 2/5
704/704 [==============================] - 21s 29ms/step - loss: 0.3701 - accuracy: 0.8362 - val_loss: 0.3181 - val_accuracy: 0.8616
Epoch 3/5
704/704 [==============================] - 21s 30ms/step - loss: 0.2998 - accuracy: 0.8724 - val_loss: 0.3706 - val_accuracy: 0.8336
Epoch 4/5
704/704 [==============================] - 23s 33ms/step - loss: 0.2722 - accuracy: 0.8887 - val_loss: 0.3342 - val_accuracy: 0.8528
Epoch 5/5
704/704 [==============================] - 23s 32ms/step - loss: 0.2543 - accuracy: 0.8957 - val_loss: 0.3388 - val_accuracy: 0.8540


## Reusing Pretrained Embeddings and Language Models
**Warning**: the following cell will take a while to run (possibly an hour if you are not using a GPU).

In [51]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 61s 76ms/step - loss: 0.3135 - accuracy: 0.8650 - val_loss: 0.2370 - val_accuracy: 0.9012
Epoch 2/10
704/704 [==============================] - 54s 77ms/step - loss: 0.0500 - accuracy: 0.9841 - val_loss: 0.3250 - val_accuracy: 0.8908
Epoch 3/10
704/704 [==============================] - 53s 75ms/step - loss: 0.0064 - accuracy: 0.9988 - val_loss: 0.3829 - val_accuracy: 0.8936
Epoch 4/10
704/704 [==============================] - 51s 73ms/step - loss: 9.2777e-04 - accuracy: 0.9998 - val_loss: 0.4297 - val_accuracy: 0.9000
Epoch 5/10
704/704 [==============================] - 52s 74ms/step - loss: 5.1042e-04 - accuracy: 1.0000 - val_loss: 0.4575 - val_accuracy: 0.8980
Epoch 6/10
704/704 [==============================] - 53s 75ms/step - loss: 4.8584e-04 - accuracy: 1.0000 - val_loss: 0.4575 - val_accuracy: 0.8988
Epoch 7/10
704/704 [==============================] - 53s 75ms/step - loss: 0.0076 - accuracy: 0.9992 - val_loss: 0.4917 - v

## An Encoder–Decoder Network for Neural Machine Translation

In [52]:

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

In [53]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [54]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [55]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [56]:

text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [57]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [58]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [59]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [60]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [61]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [62]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [63]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [64]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 36s 11ms/step - loss: 0.4185 - accuracy: 0.4219 - val_loss: 0.3142 - val_accuracy: 0.5165
Epoch 2/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.2708 - accuracy: 0.5654 - val_loss: 0.2429 - val_accuracy: 0.5974
Epoch 3/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.2126 - accuracy: 0.6370 - val_loss: 0.2099 - val_accuracy: 0.6437
Epoch 4/10
3125/3125 [==============================] - 33s 10ms/step - loss: 0.1788 - accuracy: 0.6835 - val_loss: 0.1948 - val_accuracy: 0.6647
Epoch 5/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.1549 - accuracy: 0.7175 - val_loss: 0.1887 - val_accuracy: 0.6732
Epoch 6/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.1358 - accuracy: 0.7458 - val_loss: 0.1858 - val_accuracy: 0.6785
Epoch 7/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.1196 - accuracy: 0.7709 - val_loss: 0.1867 -

In [65]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input 
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [66]:
translate("I like soccer")

1/1 [==============================] - 0s 12ms/step


'me gusta el fútbol'

In [67]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 12ms/step


'me gusta la playa y mañana los domingos'

Nice! However, the model struggles with longer sentences:

## Bidirectional RNNs

To create a bidirectional recurrent layer, just wrap a regular recurrent layer in a Bidirectional layer:

In [68]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

In [69]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [70]:
# extra code — completes the model and trains it
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 52s 15ms/step - loss: 0.3081 - accuracy: 0.5390 - val_loss: 0.2188 - val_accuracy: 0.6321
Epoch 2/10
3125/3125 [==============================] - 44s 14ms/step - loss: 0.1909 - accuracy: 0.6700 - val_loss: 0.1872 - val_accuracy: 0.6735
Epoch 3/10
3125/3125 [==============================] - 45s 14ms/step - loss: 0.1594 - accuracy: 0.7136 - val_loss: 0.1751 - val_accuracy: 0.6923
Epoch 4/10
3125/3125 [==============================] - 45s 14ms/step - loss: 0.1387 - accuracy: 0.7429 - val_loss: 0.1707 - val_accuracy: 0.7010
Epoch 5/10
3125/3125 [==============================] - 43s 14ms/step - loss: 0.1219 - accuracy: 0.7678 - val_loss: 0.1700 - val_accuracy: 0.7032
Epoch 6/10
3125/3125 [==============================] - 42s 14ms/step - loss: 0.1076 - accuracy: 0.7908 - val_loss: 0.1732 - val_accuracy: 0.6999
Epoch 7/10
3125/3125 [==============================] - 41s 13ms/step - loss: 0.0952 - accuracy: 0.8107 - val_loss: 0.1764 -

## Beam Search
This is a very basic implementation of beam search. I tried to make it readable and understandable, but it's definitely not optimized for speed! The function first uses the model to find the top k words to start the translations (where k is the beam width). For each of the top k translations, it evaluates the conditional probabilities of all possible words it could add to that translation. These extended translations and their probabilities are added to the list of candidates. Once we've gone through all top k translations and all words that could complete them, we keep only the top k candidates with the highest probability, and we iterate over and over until they all finish with an EOS token. The top translation is then returned (after removing its EOS token).

Note: If p(S) is the probability of sentence S, and p(W|S) is the conditional probability of the word W given that the translation starts with S, then the probability of the sentence S' = concat(S, W) is p(S') = p(S) * p(W|S). As we add more words, the probability gets smaller and smaller. To avoid the risk of it getting too small, which could cause floating point precision errors, the function keeps track of log probabilities instead of probabilities: recall that log(a*b) = log(a) + log(b), therefore log(p(S')) = log(p(S)) + log(p(W|S)).

In [71]:
# extra code – a basic implementation of beam search

def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # encoder input
    X_dec = np.array(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]
    
    # extra code – displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = np.array([sentence_en])  # encoder input
            X_dec = np.array(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # extra code – displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [72]:
# extra code – shows how the model making an error
sentence_en = "I love cats and dogs"
translate(sentence_en)

1/1 [==============================] - 0s 22ms/step


'me [UNK] los gatos y los perros'

In [73]:
# extra code – shows how beam search can help
beam_search(sentence_en, beam_width=3, verbose=True)

1/1 [==============================] - 0s 15ms/step
Top first words: [(-0.013778946, 'me'), (-4.927514, '[UNK]'), (-5.730298, 'yo')]
1/1 [==============================] - 0s 13ms/step
Top translations so far: [(-0.48390177, 'me [UNK]'), (-1.2971523, 'me encanta'), (-2.986138, 'me gustan')]
1/1 [==============================] - 0s 15ms/step
Top translations so far: [(-0.9888767, 'me [UNK] los'), (-1.8014625, 'me [UNK] y'), (-2.3677027, 'me encanta el')]
1/1 [==============================] - 0s 14ms/step
Top translations so far: [(-0.9895286, 'me [UNK] los gatos'), (-1.9023896, 'me [UNK] y los'), (-3.3504558, 'me encanta el amor')]
1/1 [==============================] - 0s 14ms/step
Top translations so far: [(-1.2104955, 'me [UNK] los gatos y'), (-1.9268587, 'me [UNK] y los gatos'), (-4.1639295, 'me [UNK] los gatos tanto')]
1/1 [==============================] - 0s 16ms/step
Top translations so far: [(-1.4070098, 'me [UNK] los gatos y los'), (-2.4071856, 'me [UNK] y los gatos son'), (

'me [UNK] los gatos y los perros'

The correct translation is in the top 3 sentences found by beam search, but it's not the first. Since we're using a small vocabulary, the [UNK] token is quite frequent, so you may want to penalize it (e.g., divide its probability by 2 in the beam search function): this will discourage beam search from using it too much.

## Attention Mechanisms
We need to feed all the encoder's outputs to the ```Attention``` layer, so we must add ```return_sequences=True``` to the encoder:

In [74]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

In [75]:
# extra code – this part of the model is exactly the same as earlier
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

And finally, let's add the ```Attention``` layer and the output layer:

In [76]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [77]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 54s 16ms/step - loss: 0.3184 - accuracy: 0.5370 - val_loss: 0.2153 - val_accuracy: 0.6462
Epoch 2/10
3125/3125 [==============================] - 57s 18ms/step - loss: 0.1941 - accuracy: 0.6768 - val_loss: 0.1894 - val_accuracy: 0.6823
Epoch 3/10
3125/3125 [==============================] - 55s 18ms/step - loss: 0.1690 - accuracy: 0.7100 - val_loss: 0.1785 - val_accuracy: 0.6990
Epoch 4/10
3125/3125 [==============================] - 51s 16ms/step - loss: 0.1529 - accuracy: 0.7318 - val_loss: 0.1740 - val_accuracy: 0.7061
Epoch 5/10
3125/3125 [==============================] - 44s 14ms/step - loss: 0.1397 - accuracy: 0.7500 - val_loss: 0.1726 - val_accuracy: 0.7088
Epoch 6/10
3125/3125 [==============================] - 44s 14ms/step - loss: 0.1284 - accuracy: 0.7665 - val_loss: 0.1746 - val_accuracy: 0.7083
Epoch 7/10
3125/3125 [==============================] - 44s 14ms/step - loss: 0.1190 - accuracy: 0.7802 - val_loss: 0.1766 -